#Final B3 - Grupo 1

Análisis de DF de clientes bancarios para detectar bajas en un periodo de dos meses aplicando modelos predictivos.

Notebook 1/3 = Preparación

Integrantes:

Pascuccio, José (1017995)

Bugarin Carlos Maximiliano (82125)

Calda Ariel Gustavo (1158887)

Capello Juan Andrés (1158799)

Ricci Cecilia (1160479)

Siracusa Angel Jonatán (1160480)

Nievas Ivana (1157295)


##**Importación de librerías**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/2. UADE Data Science Course/@3. Machine Learning/Final'


Mounted at /content/drive


## **Carga inicial del DF**

In [3]:
df = pd.read_csv(path +'/MESES_hasta_201404.zip')


FileNotFoundError: ignored

## **Limpieza inicial sobre el csv**

In [ ]:
# Filtro los 3 últimos meses
df2=df[df.foto_mes > 201401].copy()
df2.head()

## **Tratamiento de columnas fecha**

In [ ]:
mask = ((df2['foto_mes'].astype('str')==df2['Master_fechaalta'].str[:6]) | (df2['foto_mes'].astype('str')==df2['Visa_fechaalta'].str[:6])) 
df2['alta_tarj_mes'] = np.where(mask, 1, 0)
df2 = df2.drop(['Master_fechaalta','Visa_fechaalta','Visa_Fvencimiento', 'Master_Fvencimiento'], axis='columns')

In [ ]:
mask = (df2['Master_fultimo_cierre']=='.')
print(df2[mask]['Master_fultimo_cierre'].head(10))
df2.loc[df2[mask].index, ['Master_fultimo_cierre']]=df2[mask]['foto_mes']*100 + 15
print(df2[mask]['Master_fultimo_cierre'].head(10))

In [ ]:
mask = (df2['Visa_fultimo_cierre']=='.')
print(df2[mask]['Visa_fultimo_cierre'].head(10))
df2.loc[df2[mask].index, ['Visa_fultimo_cierre']]=df2[mask]['foto_mes']*100 + 15
print(df2[mask]['Visa_fultimo_cierre'].head(10))

## **Creación de variable objetivo**

In [ ]:
# Creo un lag (lo voy a usar para crear la variable objetivo)
df2['Mes_1'] = df2.groupby(['numero_de_cliente'])['foto_mes'].shift(-1)
df2['Mes_2'] = df2.groupby(['numero_de_cliente'])['foto_mes'].shift(-2)

In [ ]:
# Despues de crear el lag puedo descartar foto_mes > 201402
df2=df2[df2.foto_mes == 201402]

In [ ]:
#Verifico el formato de la tabla
df2[['numero_de_cliente', 'foto_mes', 'Mes_1', 'Mes_2']].head(10)

In [ ]:
df2['target'] = "Continua"
df2.loc[(df2['Mes_2'].isnull()), 'target'] = "Baja+2"
df2.loc[(df2['Mes_1'].isnull()), 'target'] = "Baja+1"

In [ ]:
df2['target_bin'] = np.where(df2['target']=='Baja+2', 1, 0)

In [ ]:
df2[df2['target_bin']==1]['numero_de_cliente'].count()

In [ ]:
print(df2.shape)
df2 = df2.drop(['Mes_1' , 'Mes_2', 'target'] , axis='columns')
print(df2.shape)

### Limpieza completa del CSV

#### Chequeamos que columnas tienen valores iguales a lo largo de todo el dataset

In [ ]:
df2['marketing_activo_ultimos90dias'] = df2['marketing_activo_ultimos90dias'].astype(int)

In [ ]:
for c in df2.columns:
    if len(df2[c].unique())==1:
        print(f'{c}={df2[c].unique()}')

In [ ]:
col_drop_1 = []
for c in df2.columns:
    if ((c!='foto_mes') & (len(df2[c].unique())==1)):
        col_drop_1.append(c)

print(df2.shape) 
df2 = df2.drop(col_drop_1 , axis='columns')
print(f'Eliminadas columnas: {col_drop_1}')        
print(df2.shape)  

In [ ]:
for c in df2.columns:
    a = df2[c].unique().tolist()
    if (len(a)==2) & (a.count('S')==1):
        print(f'{c}={a}')


In [ ]:
# Esto solo se puede hacer en las columnas binarias (de 2 valores)
for c in df2.columns:
    a = df2[c].unique().tolist()
    if (len(a)==2) & (a.count('S')==1):
        df2[c] = np.where(df2[c]=='S', 1, 0)

In [ ]:
#Chequeamos lo modificado
for c in df2.columns:
    a = df2[c].unique().tolist()
    if len(a)==2:
        print(f'{c}={a}')

In [ ]:
columnas_limpiar = pd.DataFrame(df2)
missings_columnas_limpiar = columnas_limpiar.isnull().sum() * 100 / len(columnas_limpiar)
listado_missing_columnas_limpiar = pd.DataFrame({'column_name': columnas_limpiar.columns,
                                 'missings_columnas_limpiar': missings_columnas_limpiar})
listado_missing_columnas_limpiar.sort_values('missings_columnas_limpiar', ascending = False, inplace=True)
listado_missing_columnas_limpiar.head()

#### Verificamos las columnas nulas

In [ ]:
nulos=df2.isnull().sum()
np.round((nulos[nulos>1]/len(df2))*100,2)

In [ ]:
# Eliminamos estas dos columnas
# Master_Finiciomora  99.24% de NaN
# Visa_Finiciomora    99.14% de NaN
df2 = df2.drop(['Master_Finiciomora' , 'Visa_Finiciomora'] , axis='columns')

In [ ]:
for c in df2.columns:
    if len(df2[c].unique())==1:
        print(f'{c}={df2[c].unique()}')

In [ ]:
df2.head(5)

In [ ]:
df2.shape

In [ ]:
with pd.option_context('display.max_rows', len(df2.columns.values), 'display.max_columns', 1):
    display(df2.dtypes)

## **Transformación del DF**

In [ ]:
print(df2.groupby(['Master_marca_atraso'])['Master_marca_atraso'].count())
print(df2.groupby(['Master_cuenta_estado'])['Master_cuenta_estado'].count())
print(df2.groupby(['Visa_marca_atraso'])['Visa_marca_atraso'].count())
print(df2.groupby(['Visa_cuenta_estado'])['Visa_cuenta_estado'].count())

In [ ]:
print(df2.shape)        
df2= pd.get_dummies(df2, columns=["Master_marca_atraso","Master_cuenta_estado", \
                                                      "Visa_marca_atraso","Visa_cuenta_estado"], \
                              prefix=["MMA","MCE","VMA","VCE"])
print(df2.shape)

In [ ]:
print(df2.shape)   
drop_cols= ['MMA_.', 'MCE_.','VMA_.','VCE_.']
df2 = df2.drop(drop_cols , axis='columns')
print(df2.shape) 

In [ ]:
df2['mprestamos_hipotecarios'].replace({".": "0"}, inplace=True)
df2['mcuenta_debitos_automaticos'].replace({".": "0"}, inplace=True)
df2['mttarjeta_visa_debitos_automatic'].replace({".": "0"}, inplace=True)
df2['mttarjeta_master_debitos_automat'].replace({".": "0"}, inplace=True)
df2['mpagodeservicios'].replace({".": "0"}, inplace=True)
df2['mpagomiscuentas'].replace({".": "0"}, inplace=True)
df2['Master_mfinanciacion_limite'].replace({".": "0"}, inplace=True)
df2['Master_msaldototal'].replace({".": "0"}, inplace=True)
df2['Master_msaldopesos'].replace({".": "0"}, inplace=True)
df2['Master_msaldodolares'].replace({".": "0"}, inplace=True)
df2['Master_mconsumospesos'].replace({".": "0"}, inplace=True)
df2['Master_mconsumosdolares'].replace({".": "0"}, inplace=True)
df2['Master_mlimitecompra'].replace({".": "0"}, inplace=True)
df2['Master_madelantopesos'].replace({".": "0"}, inplace=True)
df2['Master_madelantodolares'].replace({".": "0"}, inplace=True)
df2['Master_mpagado'].replace({".": "0"}, inplace=True)
df2['Master_mpagospesos'].replace({".": "0"}, inplace=True)
df2['Master_mpagosdolares'].replace({".": "0"}, inplace=True)
df2['Master_mconsumototal'].replace({".": "0"}, inplace=True)
df2['Master_mpagominimo'].replace({".": "0"}, inplace=True)
df2['Visa_mfinanciacion_limite'].replace({".": "0"}, inplace=True)
df2['Visa_msaldototal'].replace({".": "0"}, inplace=True)
df2['Visa_msaldopesos'].replace({".": "0"}, inplace=True)
df2['Visa_msaldodolares'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumospesos'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumosdolares'].replace({".": "0"}, inplace=True)
df2['Visa_mlimitecompra'].replace({".": "0"}, inplace=True)
df2['Visa_madelantopesos'].replace({".": "0"}, inplace=True)
df2['Visa_madelantodolares'].replace({".": "0"}, inplace=True)
df2['Visa_mpagado'].replace({".": "0"}, inplace=True)
df2['Visa_mpagospesos'].replace({".": "0"}, inplace=True)
df2['Visa_mpagosdolares'].replace({".": "0"}, inplace=True)
df2['Visa_mconsumototal'].replace({".": "0"}, inplace=True)
df2['Visa_mpagominimo'].replace({".": "0"}, inplace=True)
df2['mtarjeta_visa_descuentos'].replace({".": "0"}, inplace=True)
df2['mtarjeta_master_descuentos'].replace({".": "0"}, inplace=True)


In [ ]:
buscaPunto =df2.isin(['.']).any()
TienenPunto = buscaPunto[buscaPunto == True]
print(TienenPunto)

In [ ]:
 df2.select_dtypes('object').columns

In [ ]:
df2['Master_mfinanciacion_limite'] = df2['Master_mfinanciacion_limite'].astype(float)
df2['Master_msaldototal'] = df2['Master_msaldototal'].astype(float)
df2['Master_msaldopesos'] = df2['Master_msaldopesos'].astype(float)
df2['Master_msaldodolares'] = df2['Master_msaldodolares'].astype(float)
df2['Master_mconsumospesos'] = df2['Master_mconsumospesos'].astype(float)
df2['Master_mconsumosdolares'] = df2['Master_mconsumosdolares'].astype(float)
df2['Master_mlimitecompra'] = df2['Master_mlimitecompra'].astype(float)
df2['Master_madelantopesos'] = df2['Master_madelantopesos'].astype(float)
df2['Master_madelantodolares'] = df2['Master_madelantodolares'].astype(float)
df2['Master_mpagado'] = df2['Master_mpagado'].astype(float)
df2['Master_mpagospesos'] = df2['Master_mpagospesos'].astype(float)
df2['Master_mpagosdolares'] = df2['Master_mpagosdolares'].astype(float)
df2['Master_mconsumototal'] = df2['Master_mconsumototal'].astype(float)
df2['Master_mpagominimo'] = df2['Master_mpagominimo'].astype(float)
df2['Visa_mfinanciacion_limite'] = df2['Visa_mfinanciacion_limite'].astype(float)
df2['Visa_msaldototal'] = df2['Visa_msaldototal'].astype(float)
df2['Visa_msaldopesos'] = df2['Visa_msaldopesos'].astype(float)
df2['Visa_msaldodolares'] = df2['Visa_msaldodolares'].astype(float)
df2['Visa_mconsumospesos'] = df2['Visa_mconsumospesos'].astype(float)
df2['Visa_mconsumosdolares'] = df2['Visa_mconsumosdolares'].astype(float)
df2['Visa_mlimitecompra'] = df2['Visa_mlimitecompra'].astype(float)
df2['Visa_madelantopesos'] = df2['Visa_madelantopesos'].astype(float)
df2['Visa_madelantodolares'] = df2['Visa_madelantodolares'].astype(float)
df2['Visa_mpagado'] = df2['Visa_mpagado'].astype(float)
df2['Visa_mpagospesos'] = df2['Visa_mpagospesos'].astype(float)
df2['Visa_mpagosdolares'] = df2['Visa_mpagosdolares'].astype(float)
df2['Visa_mconsumototal'] = df2['Visa_mconsumototal'].astype(float)
df2['Visa_mpagominimo'] = df2['Visa_mpagominimo'].astype(float)
df2['mprestamos_hipotecarios'] = df2['mprestamos_hipotecarios'].astype(float) 
df2['mcuenta_debitos_automaticos'] = df2['mcuenta_debitos_automaticos'].astype(float)
df2['mttarjeta_visa_debitos_automatic'] = df2['mttarjeta_visa_debitos_automatic'].astype(float) 
df2['mttarjeta_master_debitos_automat'] = df2['mttarjeta_master_debitos_automat'].astype(float)
df2['mpagodeservicios'] = df2['mpagodeservicios'].astype(float) 
df2['mpagomiscuentas'] = df2['mpagomiscuentas'].astype(float) 
df2['mtarjeta_visa_descuentos'] = df2['mtarjeta_visa_descuentos'].astype(float)
df2['mtarjeta_master_descuentos'] = df2['mtarjeta_master_descuentos'].astype(float)


In [ ]:
 df2.select_dtypes('object').columns

In [ ]:
df2 = df2.drop(['Master_fultimo_cierre' , 'Visa_fultimo_cierre'] , axis='columns')
print(df2.shape) 


In [ ]:
with pd.option_context('display.max_rows', len(df2.columns.values), 'display.max_columns', 1):
    display(df2.dtypes)


In [ ]:
df2.head(5)


In [ ]:
print(df2.shape)

**Conclusión Limpieza DF**

Luego de limpiar el DF, salvar todos los registros nulos y borrar columnas que consideramos que no aportaban información valiosa para este estudio guardamos este DF de 150 columnas para usarlo posteriormente en otros notebooks.

In [ ]:
df150 = df2.copy()

# Guardamos este DF de 150 Columnas para levantarlo después desde otro notebook
df2.to_csv(path + '/DF150.csv')


## **Selección y extracción de columnas (Feature selection / extration)**

Tomando como base el DF anterior de 150 columnas vemos de limitar el número de columnas perdiendo poco o nada de información. Sospechamos que un menor número de columnas acelerará el entrenamiento de los diferentes modelos. 

### **Selección de columnas principales vía RFE**

Vamos a entrenar un Random Forest con el estimador RFE (Recursive Feature Extractor) y que el modelo nos diga qué columnas son las que considera más importantes.

### **Creamos los set de train y test**

In [ ]:
X = df2.drop(['target_bin'], axis='columns')
y = df2.target_bin

print(X.shape)
print(y.shape)

In [ ]:
# Hacemos un Random Forest con Cross Validation
rf = RandomForestClassifier(n_estimators=5, max_depth=7, random_state=1)
rfecv = RFECV(estimator=rf, step=7, min_features_to_select=70, cv=StratifiedKFold(5), scoring='roc_auc', verbose=1)

rfecv.fit(X,y)


In [ ]:
rfecv.n_features_


In [ ]:
# Listado de columnas importantes seleccionadas por el RFE
listacol = X.columns.tolist()
for i in range(X.shape[1]):
  print('Column %d: %s, Selected: %s, Rank: %.d' % (i, listacol[i], rfecv.support_[i], rfecv.ranking_[i]))

**Conclusión Feature Extration con RFE**

El RFE seleccionas muchas columnas como importantes: más de 100 Columnas (dependiendo de la corrida ya que hay una componente aleatoria en cómo se contruyen los Random Forest). Esto no baja sensiblemente la cantidad de columnas por lo que buscaremos otra estrategia de selección.

### **Selección y modificación de columnas con PCA**

Vamos a usar PCA (Principal Components Analysis) para ver cuantos componentes se necesitan para conservar el 95% de la información.

In [ ]:
# Antes de aplicar PCA se escala en DF usando StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)


In [ ]:
# Aplicamos PCA al array escalado
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_scaled)


In [ ]:
# Vemos cuantos componentes necesitamos para retener el 95% de la info
suma = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(suma >= 0.95)
print('Componentes necesarios para 95% Varianza: ' + str(d))

plt.plot(range(0,149),np.cumsum(pca.explained_variance_ratio_), alpha=.8)
plt.plot(d, 0.95, 'g*', c='k')
plt.xlabel('Componentes')
plt.ylabel('Varianza')
plt.grid(True)
plt.title("Varianza acumulada")
plt.show()


**Conclusión Feature Extration con PCA**

PCA nos dice que necesitamos al menos 100 componentes para retener el 95% de la información. Nuevamente esto no baja sensiblemente la cantidad de columnas por lo que buscaremos otra estrategia de selección.

### **Selección y modificación de columnas manualmente**

Vamos a achicar el número de columnas del DF manualmente. Generaremos columnas que sumen varias columnas originales (y borraremos estas), eliminaremos las columnas categóricas (ej.: Tiene tarjeta Visa) ya que los montos asociados explican si tiene tarjeta o no. Esperamos retener la mayor cantidad posible de información con muchas menos columnas.

In [ ]:
df2 = df2.drop(['mrentabilidad', 'marketing_coss_selling', 'tpaquete2', 'tpaquete4', 'tpaquete6', 'tpaquete7', 'tpaquete9'] , axis='columns')
df2 = df2.drop(['tcuentas', 'tcuenta_corriente', 'tcaja_ahorro', 'ttarjeta_debito', 'ttarjeta_visa', 'ttarjeta_master'] , axis='columns')
df2 = df2.drop(['cprestamos_personales', 'cprestamos_prendarios', 'cprestamos_hipotecarios', 'tplazo_fijo', 'tfondos_comunes_inversion'] , axis='columns')
df2 = df2.drop(['ttitulos', 'ttarjeta_visa_debitos_automatico', 'ttarjeta_master_debitos_automati', 'tpagodeservicios', 'tpagomiscuentas'] , axis='columns')
df2 = df2.drop(['ccajeros_propios_descuentos', 'ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos', 'ccomisiones_mantenimiento'] , axis='columns')
df2 = df2.drop(['ccomisiones_otras', 'tcambio_monedas', 'ccambio_monedas_compra', 'ccambio_monedas_venta'] , axis='columns')
df2 = df2.drop(['ctransferencias_recibidas', 'ctransferencias_emitidas', 'cextraccion_autoservicio', 'ccheques_depositados'] , axis='columns')
df2 = df2.drop(['ccheques_emitidos', 'ccheques_depositados_rechazados', 'ccheques_emitidos_rechazados', 'tcallcenter'] , axis='columns')


In [ ]:
df2 = df2.drop(['ccajeros_propio_transacciones', 'ccajeros_ajenos_transacciones', 'tcajas', 'tcajas_depositos'] , axis='columns')
df2 = df2.drop(['tcajas_extracciones', 'tcajas_otras', 'Master_msaldopesos', 'Master_msaldodolares', 'Visa_msaldopesos'] , axis='columns')
df2 = df2.drop(['Visa_msaldodolares', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_mconsumototal'] , axis='columns')
df2 = df2.drop(['Master_tconsumos', 'Master_tadelantosefectivo', 'Master_mpagominimo', 'Visa_mpagado', 'Visa_mpagospesos'] , axis='columns')
df2 = df2.drop(['Visa_mpagosdolares', 'Visa_mconsumototal', 'Visa_tconsumos', 'Visa_tadelantosefectivo', 'Visa_mpagominimo', 'alta_tarj_mes'] , axis='columns')
print(df2.shape) 


In [ ]:
# Creo variables sumando items similares y borrando las columnas originales

df2['mcuentas_all'] = (df2['mcuenta_corriente_Nopaquete'] + df2['mcuenta_corriente_Paquete']
                       + df2['mcaja_ahorro_Paquete'] + df2['mcaja_ahorro_Nopaquete'] + 8 * df2['mcaja_ahorro_dolares']
                       + df2['mplazo_fijo_pesos'] + 8 * df2['mplazo_fijo_dolares'])
df2 = df2.drop(['mcuenta_corriente_Nopaquete', 'mcuenta_corriente_Paquete', 'mcaja_ahorro_Paquete', 'mcaja_ahorro_Nopaquete'], axis='columns')
df2 = df2.drop(['mcaja_ahorro_dolares', 'mplazo_fijo_pesos', 'mplazo_fijo_dolares'], axis='columns')

df2 = df2.drop(['ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones'], axis='columns')
df2 = df2.drop(['Master_mconsumospesos', 'Master_mconsumosdolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares'], axis='columns')

df2['mtarjeta_consumo_all'] = df2['mtarjeta_visa_consumo'] + df2['mtarjeta_master_consumo']
df2 = df2.drop(['mtarjeta_visa_consumo', 'mtarjeta_master_consumo'], axis='columns')

df2['mprestamos_all'] = df2['mprestamos_personales'] + df2['mprestamos_prendarios'] + df2['mprestamos_hipotecarios']
df2 = df2.drop(['mprestamos_personales', 'mprestamos_prendarios', 'mprestamos_hipotecarios'], axis='columns')

df2['mfondos_inversion_all'] = df2['mfondos_comunes_inversion_pesos'] + 8 * df2['mfondos_comunes_inversion_dolare'] + df2['mtitulos']
df2 = df2.drop(['mfondos_comunes_inversion_pesos', 'mfondos_comunes_inversion_dolare', 'mtitulos'], axis='columns')

df2['mpagos_all'] = df2['mpagodeservicios'] + df2['mpagomiscuentas']
df2 = df2.drop(['mpagodeservicios', 'mpagomiscuentas'], axis='columns')

df2['mdebitos_automaticos_all'] = df2['mcuenta_debitos_automaticos'] + df2['mttarjeta_visa_debitos_automatic'] + df2['mttarjeta_master_debitos_automat']
df2 = df2.drop(['mcuenta_debitos_automaticos', 'mttarjeta_visa_debitos_automatic', 'mttarjeta_master_debitos_automat'], axis='columns')
df2 = df2.drop(['mplan_sueldo_manual', 'cplan_sueldo_transaccion', 'tcuenta_debitos_automaticos'], axis='columns')

df2['mdescuentos_all'] = df2['mcajeros_propios_descuentos'] + df2['mtarjeta_visa_descuentos'] + df2['mtarjeta_master_descuentos']
df2 = df2.drop(['mcajeros_propios_descuentos', 'mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'], axis='columns')

df2['mcomisiones_all'] = df2['mcomisiones_mantenimiento'] + df2['mcomisiones_otras']
df2 = df2.drop(['mcomisiones_mantenimiento', 'mcomisiones_otras'], axis='columns')

df2['mcambio_monedas_all'] = df2['mcambio_monedas_compra'] + df2['mcambio_monedas_venta']
df2 = df2.drop(['mcambio_monedas_compra', 'mcambio_monedas_venta'], axis='columns')

df2['mtransferencias_all'] = df2['mtransferencias_recibidas'] + df2['mtransferencias_emitidas']
df2 = df2.drop(['mtransferencias_recibidas', 'mtransferencias_emitidas'], axis='columns')

df2['mcheques_all'] = df2['mcheques_depositados'] + df2['mcheques_emitidos']
df2 = df2.drop(['mcheques_depositados', 'mcheques_emitidos'], axis='columns')

df2['mcheques_rechazados_all'] = df2['mcheques_depositados_rechazados'] + df2['mcheques_emitidos_rechazados']
df2 = df2.drop(['mcheques_depositados_rechazados', 'mcheques_emitidos_rechazados'], axis='columns')

df2['mcajeros_all'] = df2['mcajeros_propio'] + df2['mcajeros_ajenos']
df2 = df2.drop(['mcajeros_propio', 'mcajeros_ajenos'], axis='columns')

df2['mfinanciacion_limite_all'] = df2['Master_mfinanciacion_limite'] + df2['Visa_mfinanciacion_limite']
df2 = df2.drop(['Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite'], axis='columns')

df2['mtarjeta_saldototal_all'] = df2['Master_msaldototal'] + df2['Visa_msaldototal']
df2 = df2.drop(['Master_msaldototal', 'Visa_msaldototal'], axis='columns')

df2['mlimitecompra_all'] = df2['Master_mlimitecompra'] + df2['Visa_mlimitecompra']
df2 = df2.drop(['Master_mlimitecompra', 'Visa_mlimitecompra'], axis='columns')

df2['mtarjeta_adelanto_all'] = df2['Master_madelantopesos'] + df2['Visa_madelantopesos'] + 8 * df2['Master_madelantodolares'] + 8 * df2['Visa_madelantodolares']
df2 = df2.drop(['Master_madelantopesos', 'Visa_madelantopesos', 'Master_madelantodolares', 'Visa_madelantodolares'], axis='columns')

print(df2.shape) 

In [ ]:
df2 = df2.drop(['MMA_0', 'MMA_1', 'MCE_10', 'MCE_11', 'MCE_12', 'MCE_19'] , axis='columns')
df2 = df2.drop(['VMA_0', 'VMA_1', 'VCE_10', 'VCE_11', 'VCE_12', 'VCE_19'] , axis='columns')
print(df2.shape) 


In [ ]:
df2['tseguro_vida_mercado_abierto'] = df2['tseguro_vida_mercado_abierto'].astype(bool)
df2['tseguro_auto'] = df2['tseguro_auto'].astype(bool)
df2['tseguro_vivienda'] = df2['tseguro_vivienda'].astype(bool)
df2['tseguro_accidentes_personales'] = df2['tseguro_accidentes_personales'].astype(bool)
df2['tseguro_all'] = df2['tseguro_vida_mercado_abierto'] | df2['tseguro_auto'] | df2['tseguro_vivienda'] | df2['tseguro_accidentes_personales']
df2 = df2.drop(['tseguro_vida_mercado_abierto', 'tseguro_auto', 'tseguro_vivienda', 'tseguro_accidentes_personales'] , axis='columns')
df2['tseguro_all'] = df2['tseguro_all'].astype(int)

df2[['numero_de_cliente', 'tseguro_all']].head(20)


In [ ]:
print(df2.shape)


In [ ]:
with pd.option_context('display.max_rows', len(df2.columns.values), 'display.max_columns', 1):
    display(df2.dtypes)

**Conclusión Limpieza DF**

Se logró dejar un DF de 34 columnas con la mayor información posible. Guardamos este DF de 34 columnas para usarlo posteriormente en otros notebooks.

In [ ]:
df34 = df2.copy()

# Guardamos este DF de 34 Columnas para levantarlo después desde otro notebook
df2.to_csv(path + '/DF34.csv')


## **Test de Datasets**

Vamos a tomar los dos DF generados y verificaremos si tienen información similar. Para ello entrenamos un Random Forest con ambos DF y comparamos resultados. 

### **Creamos los set de train y test**

In [ ]:
# Ensayo el DF de 150 columnas
X = df150.drop(['target_bin'], axis='columns')
y = df150.target_bin

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape)
print(X_test.shape)


In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_depth=5, class_weight='balanced', max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)


In [ ]:
y_train_pred = rnd_clf.predict(X_train)
y_test_pred = rnd_clf.predict(X_test)

print('DF de 150 columnas')
print('Accuracy: Train: %.3f - Test: %.3f \n' % (accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)))
print('Recall: Train: %.3f - Test: %.3f \n' % (recall_score(y_train, y_train_pred), recall_score(y_test, y_test_pred)))
print('Precision: Train: %.3f - Test: %.3f \n' % (precision_score(y_train, y_train_pred), precision_score(y_test, y_test_pred)))
print('ROC AUC: Train: %.3f - Test: %.3f \n' % (roc_auc_score(y_train, y_train_pred), roc_auc_score(y_test, y_test_pred)))


In [ ]:
# Ensayo el DF de 34 columnas
X = df34.drop(['target_bin'], axis='columns')
y = df34.target_bin

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(X_train.shape)
print(X_test.shape)


In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_depth=5, class_weight='balanced', max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)


In [ ]:
y_train_pred = rnd_clf.predict(X_train)
y_test_pred = rnd_clf.predict(X_test)

print('DF de 150 columnas')
print('Accuracy: Train: %.3f - Test: %.3f \n' % (accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)))
print('Recall: Train: %.3f - Test: %.3f \n' % (recall_score(y_train, y_train_pred), recall_score(y_test, y_test_pred)))
print('Precision: Train: %.3f - Test: %.3f \n' % (precision_score(y_train, y_train_pred), precision_score(y_test, y_test_pred)))
print('ROC AUC: Train: %.3f - Test: %.3f \n' % (roc_auc_score(y_train, y_train_pred), roc_auc_score(y_test, y_test_pred)))


**Conclusión Test de Datasets**

Se entrenó el mismo modelo con ambos DF y las métricas que obtuvimos fueron muy parecidas. Concluímos que ambos DF tienen igual cantidad de información respecto del tema que queremos evaluar. Cualquiera de los dos DF puede ser usado para entrenar a los modelos.

Decidimos usar en adelante el DF de 150 columnas (sólo se lo limpió y se corrigieron los faltantes y errores) si el entrenamiento del modelo lo hace posible (computacionalmente hablando). Si se necesitara un DF más liviano se usará el DF de 34 columnas.
